#PREPARACIÓN DE DATOS

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

import plotly.express as px
import matplotlib.pyplot as plt

from glob import glob
from skimage import io
from shutil import copy

import uuid
import time
pd.options.mode.chained_assignment = None  

#import xmltodict
import glob
import xml.etree.ElementTree as ET

import random as rnd
from PIL import Image
from matplotlib import patches as mpatches

import imutils
from pathlib import Path
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', 6500)


**REPARACIÓN DE DATOS**

In [ ]:
# Contar cantidad de archivos en un directorio
#import glob

imagenes=len(glob.glob("/content/drive/MyDrive/TesisDataCarrosXML/*.jpg"))   
etiquetas=len(glob.glob("/content/drive/MyDrive/TesisDataCarrosXML/*.xml"))

print("jpg=", imagenes)
print(" xml=",etiquetas)


#1B) Extraer metadatos

In [ ]:
# producto de haber recortado las imagenes veremos las dimensiones de estas 
#guarda en Excel el {nombre,ancho,alto} de las imagenes que no estan 
carpeta = os.listdir('/content/drive/MyDrive/TesisDataFotos/')
datasetFot = {"nombre":[],"ancho":[],"alto":[]}

for pth in carpeta:
    img =cv2.imread('/content/drive/MyDrive/TesisDataFotos/' + pth)
    datasetFot['nombre'].append(pth)
    datasetFot['ancho'].append(img.shape[1])
    datasetFot['alto'].append(img.shape[0])

datasetImg = pd.DataFrame(datasetFot)
#guardamos en excel
datasetImg.to_csv('/content/drive/MyDrive/TESIS/TesisFotoNombreAnchoAlto.csv')
datasetImg.head()

In [ ]:
# revisamos la informacion
#datasetImg.info()
datasetImg.head()

#2C) Cambio de tamaño de Imagenes a 640 x 640 pixels
-Una vez que se recortaron las imagenes manualmente, redimensionamos para que se tenga de un solo tamaño.\
-Posteriormente se etiquetaran las imagenes con la herramienta LabelImg.

In [ ]:
# Cambiar tamaño de fotos 2 metodos
imgs = os.listdir('/content/drive/MyDrive/TesisDataFotos/') 

for pth in imgs:
    img =cv2.imread('/content/drive/MyDrive/TesisDataFotos/' + pth) # lee la imagen
    #img2=cv2.resize(img,(640,640), interpolation=cv2.INTER_CUBIC) # 1er metodo
    img2= imutils.resize(img, height=640) # 2do metodo
    #axi.imshow(img2, cmap = "gray")
    #cv2.imshow("original",img2)
    cv2.imwrite('/content/drive/MyDrive/TesisDataCarros/' + pth, img2) # guarda la nueva imagen en otro directorio 

# 3)INTEGRACION DE DATOS
-Una vez que se tienen las imagenes etiquetadas, creamos las anotaciones en formato (txt) con las variables normalizadas.\
-Guardamos las variables normalizadas en un archivo de CSV \
-Verificar la correctitud de las variables normalizadas.

In [ ]:
# extrrae informacion de las imagenes etiquetadas
dataset = {"imagen":[],"ancho":[],"alto":[],"xmin":[],"ymin":[],"xmax":[],"ymax":[]}
carpeta="/content/drive/MyDrive/TesisDataCarrosXML/*.xml"  

for item in glob.glob(carpeta):
    tree = ET.parse(item)
    
    for elem in tree.iter():
        if 'filename' in elem.tag:
            filename=elem.text
        elif 'width' in elem.tag:
            width=int(elem.text)
        elif 'height' in elem.tag:
            height=int(elem.text)
        elif 'xmin' in elem.tag:
            xmin=int(elem.text)
        elif 'ymin' in elem.tag:
            ymin=int(elem.text)
        elif 'xmax' in elem.tag:
            xmax=int(elem.text)
        elif 'ymax' in elem.tag:
            ymax=int(elem.text)
            
            dataset['imagen'].append(filename)
            dataset['ancho'].append(width)
            dataset['alto'].append(height)
            dataset['xmin'].append(xmin)
            dataset['ymin'].append(ymin)
            dataset['xmax'].append(xmax)
            dataset['ymax'].append(ymax)


In [ ]:
datasetXml = pd.DataFrame(dataset)

datasetXml.to_csv('/content/drive/MyDrive/TESIS/TesisDataCarrosXML.csv') #guardamos
datasetXml.head()

,imagen,ancho,alto,xmin,ymin,xmax,ymax
0,IMG_20230115_181516.jpg,1990,1990,212,666,325,740
1,IMG_20230115_181516.jpg,1990,1990,1419,701,1541,774
2,IMG_20230115_181518.jpg,2320,2320,175,629,300,707
3,IMG_20230115_181518.jpg,2320,2320,1546,673,1705,781
4,IMG_20230115_181520.jpg,2198,2198,1,683,79,760


In [ ]:
# -Verificar la correctitud de las variables.
datasetXml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1529 entries, 0 to 1528
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   imagen  1529 non-null   object
 1   ancho   1529 non-null   int64 
 2   alto    1529 non-null   int64 
 3   xmin    1529 non-null   int64 
 4   ymin    1529 non-null   int64 
 5   xmax    1529 non-null   int64 
 6   ymax    1529 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 83.7+ KB


**Normalizamos los valores de las coordenadas  de los rectangulos delimitadores**

**(que estan almacenados en formato .xml) y lo guardamos en formato .txt**

**Es decir convertimos los archivos .xml a .txt**

In [ ]:
#------------------------------------------------------------------------------------------------------------------------
datasetTxt = pd.DataFrame(dataset)
x_pos = []
y_pos = []
frame_width = []
frame_height = []

creaCarpeta = Path("/content/drive/MyDrive/TesisDataCarros")
creaCarpeta.mkdir(parents=True, exist_ok=True)
save_type = 'w'

for i, row in enumerate(datasetTxt.iloc):
  current_filename = str(row.imagen[:-4])# el 4 elimina .jpg del nombre del archivo 
  #print(current_filename)

  ancho, alto, xmin, ymin, xmax, ymax = list(datasetTxt.iloc[i][-6:])
  x=(xmin+xmax)/2/ancho
  y=(ymin+ymax)/2/alto
  ancho=(xmax-xmin)/ancho
  alto=(ymax-ymin)/alto
  x_pos.append(x)
  y_pos.append(y)
  frame_width.append(ancho)
  frame_height.append(alto)
  txt = '0' + ' ' + str(x) + ' ' + str(y) + ' ' + str(ancho) + ' ' + str(alto) + '\n'

  if i > 0:
    previous_filename = str(datasetTxt.imagen[i-1][:-4])
    save_type='a+' if current_filename == previous_filename else 'w'

  with open("/content/drive/MyDrive/TesisDataCarros/" + str(row.imagen[:-4]) +'.txt', save_type) as f:
    f.write(txt)
 

datasetTxt['x_pos']=x_pos
datasetTxt['y_pos']=y_pos
datasetTxt['f_width']=frame_width
datasetTxt['f_height']=frame_height

datasetTxt

,imagen,ancho,alto,xmin,ymin,xmax,ymax,x_pos,y_pos,f_width,f_height
0,IMG_20230115_181516.jpg,1990,1990,212,666,325,740,0.134925,0.353266,0.056784,0.037186
1,IMG_20230115_181516.jpg,1990,1990,1419,701,1541,774,0.743719,0.370603,0.061307,0.036683
2,IMG_20230115_181518.jpg,2320,2320,175,629,300,707,0.102371,0.287931,0.053879,0.033621
3,IMG_20230115_181518.jpg,2320,2320,1546,673,1705,781,0.700647,0.313362,0.068534,0.046552
4,IMG_20230115_181520.jpg,2198,2198,1,683,79,760,0.018198,0.328253,0.035487,0.035032
...,...,...,...,...,...,...,...,...,...,...,...
1524,IMG_20230115_181507.jpg,2070,2069,1853,668,1916,705,0.910386,0.331803,0.030435,0.017883
1525,IMG_20230115_181509.jpg,1480,1479,283,803,436,903,0.242905,0.576741,0.103378,0.067613
1526,IMG_20230115_181509.jpg,1480,1479,1243,548,1283,575,0.853378,0.379648,0.027027,0.018256
1527,IMG_20230115_181513.jpg,1352,1351,139,613,208,652,0.128328,0.468172,0.051036,0.028868


In [ ]:
#guardamos en archivo Excel
datasetTxt.to_csv('/content/drive/MyDrive/TESIS/TesisDataCarrosNormalizado.csv') #guardamos

In [ ]:
# -Verificamos la correctitud de las variables normalizadas.
datasetTxt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1529 entries, 0 to 1528
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   imagen    1529 non-null   object 
 1   ancho     1529 non-null   int64  
 2   alto      1529 non-null   int64  
 3   xmin      1529 non-null   int64  
 4   ymin      1529 non-null   int64  
 5   xmax      1529 non-null   int64  
 6   ymax      1529 non-null   int64  
 7   x_pos     1529 non-null   float64
 8   y_pos     1529 non-null   float64
 9   f_width   1529 non-null   float64
 10  f_height  1529 non-null   float64
dtypes: float64(4), int64(6), object(1)
memory usage: 131.5+ KB


**Verificamos graficamente en las imagenes si las variables normalizadas estan correctas**

In [ ]:
df = pd.DataFrame(dataset)
%pylab inline

def print_random_images(photos: list, n: int = 10, seed=None) -> None:
    if n > 10:
        n=10
    
    if seed:
        rnd.seed(seed)
        
    random_photos = rnd.sample(photos, n)
    
    for image_path in random_photos:
        
        with Image.open(image_path) as fd:
            fig, ax = plt.subplots()
            ax.imshow(fd)           
            ax.axis(False)
            
            for i, foto in enumerate(df.foto):
                if foto in image_path:
                    x1,y1,x2,y2=list(df.iloc[i, -4:])
                        
                    mpatch=mpatches.Rectangle((x1,y1),x2-x1,y2-y1,linewidth=1, edgecolor='b',facecolor="none",lw=2,)                    
                    ax.add_patch(mpatch)
                    rx, ry = mpatch.get_xy()
                    ax.annotate('placa', (rx, ry-2), color='blue', weight='bold', fontsize=12, ha='left', va='baseline')
                    
photos_path = "/content/drive/MyDrive/TESIS/datasC/*.jpg"  
photos_list = glob.glob(photos_path)

print_random_images(photos_list)

**Aumentacion de datos**
-Transformacion de imagenes a escala de grises

In [ ]:
# Convertir a escala de grises
dire="/content/drive/MyDrive/TESIS/grises/" 
guarda="/content/drive/MyDrive/TesisDataCarros/"
print(dire)
imgs = os.listdir(dire)

for pth in imgs:
  if (pth[-4:]==".jpg"):
    posicion= pth.index(".jpg")
    nombre= pth[0:posicion]
    image = cv2.imread(dire+pth)
    gris = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imwrite(guarda + nombre+"-gris.jpg", gris) # guarda la imagen convertida